In [3]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import numpy as np
import gradio as gr
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import preprocess_input

def flip_VGG16(image):
  model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/best_model.keras")

  def preprocess_image(image):
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0
    image = np.expand_dims(image, axis=0)
    return image

  def predict_image_type(image):
    preprocessed_image = preprocess_image(image)
    prediction = model.predict(preprocessed_image)
    predicted_class_index = np.argmax(prediction)
    class_names = ["cardboard", "glass", "metal", "paper", "plastic","trash"]
    predicted_class = class_names[predicted_class_index]
    confidence_score = prediction[0][predicted_class_index]
    return predicted_class, confidence_score

  def classify_image(image):
      return predict_image_type(image)

  return classify_image(image)

class ChannelAttention(tf.keras.layers.Layer):
    def __init__(self, reduction_ratio=16, **kwargs):
        super(ChannelAttention, self).__init__(**kwargs)
        self.reduction_ratio = reduction_ratio

    def build(self, input_shape):
        self.avg_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.max_pool = tf.keras.layers.GlobalMaxPooling2D()
        self.dense1 = tf.keras.layers.Dense(input_shape[-1] // self.reduction_ratio, activation='relu', use_bias=False)
        self.dense2 = tf.keras.layers.Dense(input_shape[-1], activation='sigmoid', use_bias=False)

    def call(self, inputs):
        avg_out = self.avg_pool(inputs)
        max_out = self.max_pool(inputs)
        avg_out = tf.expand_dims(tf.expand_dims(avg_out, 1), 1)
        max_out = tf.expand_dims(tf.expand_dims(max_out, 1), 1)

        avg_out = self.dense1(avg_out)
        max_out = self.dense1(max_out)

        avg_out = self.dense2(avg_out)
        max_out = self.dense2(max_out)

        out = avg_out + max_out
        return inputs * out  # Element-wise multiplication

class SpatialAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(SpatialAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.conv = tf.keras.layers.Conv2D(1, (7, 7), strides=1, padding='same', activation='sigmoid', use_bias=False)

    def call(self, inputs):
        avg_out = tf.reduce_mean(inputs, axis=-1, keepdims=True)
        max_out = tf.reduce_max(inputs, axis=-1, keepdims=True)
        concat = tf.concat([avg_out, max_out], axis=-1)
        out = self.conv(concat)
        return inputs * out  # Element-wise multiplication

class CBAM(tf.keras.layers.Layer):
    def __init__(self, reduction_ratio=16, **kwargs):
        super(CBAM, self).__init__(**kwargs)
        self.channel_attention = ChannelAttention(reduction_ratio)
        self.spatial_attention = SpatialAttention()

    def call(self, inputs):
        x = self.channel_attention(inputs)
        x = self.spatial_attention(x)
        return x

def flip_Resnet50(image):
  custom_objects = {
        'CBAM': CBAM,
        'ChannelAttention': ChannelAttention,
        'SpatialAttention': SpatialAttention }
  model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/trashnet_model_CBAM.h5", custom_objects=custom_objects)

  def preprocess_image(image):
    image = tf.image.resize(image, (224, 224))  # Resize to match model input size
    image = tf.cast(image, tf.float32)  # Convert to float32
    image = preprocess_input(image)  # Apply ResNet50-specific preprocessing
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

  def predict_image_type(image):
      preprocessed_image = preprocess_image(image)
      prediction = model.predict(preprocessed_image)
      predicted_class_index = np.argmax(prediction)  # Get the index of the highest probability
      class_names = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]
      predicted_class = class_names[predicted_class_index]
      confidence_score = prediction[0][predicted_class_index]
      return predicted_class, confidence_score

  def classify_image(image):
      return predict_image_type(image)
  return classify_image(image)

def custom_html_head():
    return """
    <title>Waste identification and sorting website</title>
    """

with gr.Blocks() as showcase:
    title_html = '<h2 style="text-align: center;">Garbage Classification System</h2>'
    gr.HTML(title_html)
    gr.Markdown("hello! please drag in or take any picture!")
    with gr.Tab("VGG16"):
        with gr.Row():
          with gr.Column():
            V_input = gr.Image(label="User upload")
            with gr.Row():
              V_button_clear = gr.Button("Clear")
              V_button_submit = gr.Button("Submit")
          with gr.Column():
            V_output_class = gr.Label(label="Class name")
            V_output_score = gr.Text(label="Accuarcy")
    with gr.Tab("Resnet50"):
        with gr.Row():
          with gr.Column():
            R_input = gr.Image(label="User upload")
            with gr.Row():
              R_button_clear = gr.Button("Clear")
              R_button_submit = gr.Button("Submit")
          with gr.Column():
            R_output_class = gr.Label(label="Class name")
            R_output_score = gr.Text(label="Accuarcy")
    with gr.Accordion("Open to know more!", open=False):
        gr.Markdown("With the rapid growth of waste production, effective waste classification has become a key issue in sustainable urban development.")
        gr.Markdown("To better solve this problem, we design a computer vision website is introduced to automatically identify and classify garbage using deep learning models to improve the intelligence and automation of garbage disposal.")
        gr.Markdown("Please pay more attention to waste segregation and a sustainable planet!")

    V_button_submit.click(flip_VGG16, inputs=V_input, outputs=[V_output_class, V_output_score])
    V_button_clear.click(lambda: (None, None, None), inputs=None, outputs=[V_input, V_output_class, V_output_score])
    R_button_submit.click(flip_Resnet50, inputs=R_input, outputs=[R_output_class, R_output_score])
    R_button_clear.click(lambda: (None, None, None), inputs=None, outputs=[R_input, R_output_class, R_output_score])

showcase.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5aa8afa0f835f4d515.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
